# Optimization

## Prerequisite Code

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hyperparameters

When training a model, there are many hyperparameters that can be set. In this example, we'll define the following:
- Number of Epochs - the number times to iterate over the dataset
- Batch Size - the number of data samples seen by the model in each epoch
- Learning Rate - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop

We will optimize our model with an optimization loop. Each iteration of the optimization loop is called an epoch.

Each epoch consists of two main parts:
- The Train Loop - iterate over the training dataset and try to converge to optimal parameters.
- The Validation/Test Loop - iterate over the test dataset to check if model performance is improving.

### Loss Function


Loss function returns the error of our model. It indicates how far the model's prediction is from the actual target.

There are lots of loss functions available in the PyTorch library. For example, `nn.MSELoss` is used for regression tasks, and `nn.NLLLoss` is used for classification. `nn.CrossEntropyLoss` combines `nn.LogSoftmax` and `nn.NLLLoss`.

Model's output logits are passed to `nn.CrossEntropyLoss`, which normalizes the logits and computes the prediction error.

In [3]:
loss_fn = nn.CrossEntropyLoss()

### Optimizer

After calculating the loss, we need to adjust the parameters of the model to minimize the error. This is done by the optimizer. All optimization logic is encapsulated in the optimizer object. In this case, we will use the SGD (Stochastic Gradient Descent) optimizer. There are other optimizers available in the PyTorch library, such as ADAM and RMSProp, that work better for different kinds of models and data.

We initialize the optimizer by registering the model's parameters that need to be trained, and passing in the learning rate hyperparameter.

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

The optimizer consists of the following steps:
- Call `optimizer.zero_grad()` to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
- Backpropagate the prediction loss with a call to `loss.backwards()`. PyTorch deposits the gradients of the loss w.r.t. each parameter.
- Once we have our gradients, we call `optimizer.step()` to adjust the parameters by the gradients collected in the backward pass.

## Full Implementation

Let's implement `train_loop` and `test_loop` functions to train the model. The `train_loop` iterates over the training dataset and updates the model parameters for each batch. The `test_loop` iterates over the test dataset and evaluates the model's performance. We'll log the model's loss and accuracy in each epoch.

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301469  [   64/60000]
loss: 2.285230  [ 6464/60000]
loss: 2.271366  [12864/60000]
loss: 2.266928  [19264/60000]
loss: 2.241690  [25664/60000]
loss: 2.212071  [32064/60000]
loss: 2.222191  [38464/60000]
loss: 2.188137  [44864/60000]
loss: 2.181123  [51264/60000]
loss: 2.147171  [57664/60000]
Test Error: 
 Accuracy: 45.3%, Avg loss: 2.138286 

Epoch 2
-------------------------------
loss: 2.155084  [   64/60000]
loss: 2.138257  [ 6464/60000]
loss: 2.078044  [12864/60000]
loss: 2.097393  [19264/60000]
loss: 2.046590  [25664/60000]
loss: 1.980241  [32064/60000]
loss: 2.009609  [38464/60000]
loss: 1.928766  [44864/60000]
loss: 1.927481  [51264/60000]
loss: 1.853386  [57664/60000]
Test Error: 
 Accuracy: 53.6%, Avg loss: 1.848370 

Epoch 3
-------------------------------
loss: 1.890868  [   64/60000]
loss: 1.850506  [ 6464/60000]
loss: 1.728421  [12864/60000]
loss: 1.776289  [19264/60000]
loss: 1.672329  [25664/60000]
loss: 1.623960  [32064/600